In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np

import sys
import os

sys.path.append(os.environ['REPO_DIR'] + '/utilities')
from utilities2015 import *

import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd

In [ ]:
patch_dir = '/home/yuncong/CSHL_data_patches/'

model_dir = '/home/yuncong/jiaxu_repo/model_publish'
model_name='inception-stage1'
model_iteration = 6
model = mx.model.FeedForward.load(os.path.join(model_dir, model_name), model_iteration, ctx=mx.gpu())

In [ ]:
def interpolate_score(sample_locations_roi, probs):
    
    interpolation_xmin, interpolation_ymin = sample_locations_roi.min(axis=0)
    interpolation_xmax, interpolation_ymax = sample_locations_roi.max(axis=0)
    interpolation_w = interpolation_xmax - interpolation_xmin + 1
    interpolation_h = interpolation_ymax - interpolation_ymin + 1
    
    print sample_locations_roi.shape[0], 'samples'
    
    preds = np.argmax(probs, axis=1).astype(np.int)
        
    ##### sample_locations_roi + scores to dense_score_map #####
    
    shrink_factor = 4
    
    sample_locations_unique_xs = np.unique(sample_locations_roi[:,0])
    sample_locations_unique_ys = np.unique(sample_locations_roi[:,1])
    
    n_sample_x = sample_locations_unique_xs.size
    n_sample_y = sample_locations_unique_ys.size
    
    index_x = dict([(j,i) for i,j in enumerate(sample_locations_unique_xs)])
    index_y = dict([(j,i) for i,j in enumerate(sample_locations_unique_ys)])
    sample_location_indices = np.asarray([(index_x[x], index_y[y]) for x, y in sample_locations_roi])
    
    sample_locations_interpolatedArea_ys_matrix, \
    sample_locations_interpolatedArea_xs_matrix = np.meshgrid(range(interpolation_ymin/shrink_factor, 
                                                                    interpolation_ymax/shrink_factor), 
                                                              range(interpolation_xmin/shrink_factor, 
                                                                    interpolation_xmax/shrink_factor), 
                                                              indexing='ij')

    def generate_score_map(l):

        score_matrix = np.zeros((n_sample_x, n_sample_y))
        score_matrix[sample_location_indices[:,0], sample_location_indices[:,1]] = probs[:,l]

        spline = RectBivariateSpline(sample_locations_unique_xs/shrink_factor, 
                                     sample_locations_unique_ys/shrink_factor, 
                                     score_matrix, 
                                     bbox=[interpolation_xmin/shrink_factor, 
                                           interpolation_xmax/shrink_factor, 
                                           interpolation_ymin/shrink_factor, 
                                           interpolation_ymax/shrink_factor])

        dense_score_map = spline.ev(sample_locations_interpolatedArea_xs_matrix, sample_locations_interpolatedArea_ys_matrix)
        
        dense_score_map = resize(dense_score_map, (interpolation_h, interpolation_w))

        return dense_score_map


    import time
    t = time.time()

    dense_score_maps = Parallel(n_jobs=12)(delayed(generate_score_map)(l) for l in range(10))

    print time.time() - t
    
    for i, m in enumerate(dense_score_maps):
        dense_score_map_lossless = np.pad(m, ((interpolation_ymin, h-interpolation_ymax-1), 
                                              (interpolation_xmin, w-interpolation_xmax-1)),
                                          mode='constant', constant_values=0)
        
        np.savez_compressed(os.path.join(denseScoreMapLossless_dir, dataset+'_denseScoreMapLossless_%d'%i), 
                            dense_score_map_lossless)

In [ ]:
def compute_dense_scoremap(stack, sec, label, model):
    
    ## define grid, generate patches
    
    output_dir = os.path.join(patch_dir, '%s_byROI/%04d/roi1'%(stack, sec))
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    half_size = patch_size/2

    dm = DataManager(stack=stack, section=sec)
    dm._load_image(['rgb-jpg'])

    indices_roi = indices_allROIs_allSections[sec]['roi1']
    sample_locations_roi = sample_locations[indices_roi]
    
    for i, (x,y) in zip(indices_roi, sample_locations_roi):
        patch = dm.image_rgb_jpg[y-half_size:y+half_size, x-half_size:x+half_size]
        cv2.imwrite(os.path.join(output_dir, '%(stack)s_%(sec)04d_%(gridp_ind)08d.jpg' %\
                                 {'stack':stack, 'sec':sec, 'gridp_ind':i}), 
                    patch[..., ::-1])
        
    ## predict for the patches
    
    data_dir = os.path.join(patch_dir, '%(stack)s_byROI/%(sec)04d/roi1') % {'stack':stack,'sec': sec}
    dataset = '%(stack)s_%(sec)04d_roi1' % {'stack': stack, 'sec': sec}
    
    img_filenames = sorted(os.listdir(data_dir))
    
    with open(patch_dir+'/%(dataset)s_test.lst' % {'dataset': dataset}, 'w') as lst_file:
        for index, img_filename in enumerate(img_filenames):
            lst_file.write('%s\t0\t%s\n'%(index, img_filename))
            
    os.system(os.environ['MXNET_DIR']+'/bin/im2rec %(patch_dir)s/%(dataset)s_test.lst \
    %(data_dir)s %(patch_dir)s/%(dataset)s_test.rec' % \
              {'dataset': dataset, 'data_dir': data_dir, 'patch_dir': patch_dir})
        
    # feed the network
    batch_size = 16

    test_iter = mx.io.ImageRecordIter(
        path_imgrec = patch_dir + '/%(dataset)s_test.rec' % {'dataset': dataset},
        batch_size = batch_size,
        data_shape = (3, 224, 224),
        mean_img = os.path.join(model_dir, 'mean_224.nd'),
    )

    t = time.time()
    scores = model.predict(test_iter)
    print time.time() - t    
#     np.save(prediction_dir+'/%(dataset)s_predictions.npy'% {'dataset': dataset}, probs)
    
    ## interpolate

    interpolate_score(sample_locations_roi, scores)

In [1]:
stack = 'MD589'
first_bs_sec, last_bs_sec = section_range_lookup[stack]

NameError: name 'section_range_lookup' is not defined

In [ ]:
table_filepath = os.path.join(patch_dir, '%(stack)s_indices_allROIs_allSections.h5'%{'stack':stack})
indices_allROIs_allSections = pd.read_hdf(table_filepath, 'indices_allROIs_allSections')
grid_parameters = pd.read_hdf(table_filepath, 'grid_parameters')
patch_size, stride, w, h = grid_parameters.tolist()

In [ ]:
for sec in range(first_bs_sec, last_bs_sec+1):
    for l in labels_to_detect:
        compute_dense_scoremap(stack, sec, label=l, model=model)